# L2: Role of the Tokenizers

<p style="background-color:#edfbff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> Tokenizers are trainable components and they <b><font color="green">learn</font></b> the best <b><font color="green"><b>vocabulary</font></b> for the given training data. Several tokenizing algorithms exist. They are fully <b>deterministic</b>, contrary to the neural networks. They depend on the <b>statistics</b> of the input data. By <b>"Vocabulary"</b>, we mean the final set of <b>distinct</b> output tokens that we get after tokenizing.</p>

### Variation of tokenizers and hyperparameters used in different embedding models
The table below shows different embedding models and the tokenizing algorithm they use. <b>vocabulary size</b> is a hyperparameter
for tokenizers

<div>
<img src="img/l2_emb_models.jpg" width="500"/>
</div>

## BPE - Byte-Pair Encoding
- Divide the sentence(s) into unique characters or bytes
- The list of tokens = our <b><font color="green"><b>vocabulary</font></b>
- Add the most occuring pair of tokens from previous step to the existing list of tokens
- Treat the newly coupled token as a `new` and `separate` token
- Repeat adding newly paired tokens
- Repeat adding new tokens until we reach our <b>vocabulary size</b>

Hands on demonstration :
https://youtu.be/HEikzVL-lZU

### Tokenize a new text

After creating our vocabulary from the given corpus using the <b>BPE Tokenizer</b>, we can now tokenize new text

1. Use the <b><font color="green"><b>longest subword</b> token from the vocab
2. Match the token with the words in the given sentence starting from left
3. When the subword matches, divide the word into the format of <b>matched subword + remainder</b>
4. Match and divide one full iteration with this subword token until the end of the sentence
5. Repeat the process with the <b>next largest subword token</b>
6. Repeat until the vocabulary is exhausted
  
#### <font color="green"><b>Example : </b></font>
- Example sentence : "she walked"
- Some top tokens out of the 14 sized vocabulary "walke" "walk" "wal" "alk" <b>...</b> "e" "k" (based on the most frequent occurrences)
- We take "<font color="red"><b>walk</b></font>", compare from the left of the sentence, and find she <font color="red"><b>walk</b></font>ed
- Tokenize the word as <font color="blue"><b>|walke|</b></font> + remainder <font color="blue"><b>"d"</b></font>
- We take the next top token "walk" and find other untokenized words in the sentence
- We repeat the searching and tokenizing steps
- The remainder of the word "walked", which is <font color="blue"><b>"d"</b></font> is matched later with token "d", so it is also included in the list of tokenized words
- When the token is "<font color="red"><b>e</b></font>" and the first untokenized word comes as "<font color="red"><b>she</b></font>", the <b>s & h</b> is considered as <b>unknown</b> tokens and are ignored afterwards, so we remain with  |<font color="blue"><b>e</b></font>|
- After all the steps, we get the tokenized sentence as <font color="blue"><b>|e| |walke| |d|</b></font>

In [37]:
import warnings
warnings.filterwarnings('ignore')

In [38]:
training_data = ["walker walked a long walk",]

In [39]:
from tokenizers.trainers import BpeTrainer
from tokenizers.models import BPE
from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Whitespace

bpe_tokenizer = Tokenizer(BPE())
bpe_tokenizer.pre_tokenizer = Whitespace()

bpe_trainer = BpeTrainer(vocab_size=14)

In [40]:
bpe_tokenizer.train_from_iterator(training_data, bpe_trainer)

By running <font color="blue">bpe_tokenizer.get_vocab()</font> after tokenizer training, we can see the learned tokens from the training data. As the training process is **iterative**, we can get the order of the tokens learned one after another by looking at their corresponding **ids**

In [41]:
# To check what are the learned tokens or vocabulary after the tokenizer training
bpe_tokenizer.get_vocab()

{'k': 4,
 'al': 10,
 'l': 5,
 'o': 7,
 'wal': 11,
 'walk': 12,
 'e': 2,
 'walke': 13,
 'r': 8,
 'w': 9,
 'd': 1,
 'g': 3,
 'a': 0,
 'n': 6}

#### Encode (<font color="red">= Tokenize</font>) new texts 

This encoder only tries to exact match the tokens out of the given text

In [42]:
bpe_tokenizer.encode("walker walked a long walk").tokens

['walke', 'r', 'walke', 'd', 'a', 'l', 'o', 'n', 'g', 'walk']

In [43]:
bpe_tokenizer.encode("wlk").ids

[9, 5, 4]

In [44]:
bpe_tokenizer.encode("wlk").tokens

['w', 'l', 'k']

In [45]:
bpe_tokenizer.encode("she walked").tokens

['e', 'walke', 'd']

In [46]:
bpe_tokenizer.encode("she talks").tokens

['e', 'al', 'k']

<p style="background-color:#edfbff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> The <b>Tokenizers</b> library introduces us with different types of tokenizers - <b><font color="green"><b>BPE, WordPiece and Unigram</font></b> <br>
Tokenizer library used : <br>
https://github.com/huggingface/tokenizers <br>
The tokenizers work in the following way <br> </p>
<ul style="background-color:#edfbff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    <li style="margin-left:10px">Instantiate the required tokenizer, for example - BPE()</li>
    <li style="margin-left:10px">The tokenizer does not have any vocabulary to start with</li>
    <li style="margin-left:10px">The Whitespace() pretokenizer splits the text into words based on whitespace seperators</li>
    <li style="margin-left:10px">Use a trainer i.e - BpeTrainer</li>
    <li style="margin-left:10px">The trainer learns the vocabulary, trains the tokenizer model</li>
    <li style="margin-left:10px">Pass the trainer and training_data to the tokenizer and train using a python iterator (train_from_iterator method)</li>
<ul>

## WordPiece

<p style="background-color:#edfbff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
The only difference between <b>WordPiece</b> and <b>BPE</b> is that wordpiece differentiates between initial and middle letters / subword inside a word by adding <b>## prefix</b> to the middle tokens. The general idea of this algorithm is to learn words, prefixes and suffixes separately. The basic steps are the same as BPE, but only t
</p>

<div>
    <img src="img/l2_wordpiece_1.jpg" width=500px>   
</div>

<ul style="background-color:#fad7ac; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    <li style="margin-left:10px">Break down the corpus into <b>distinct</b> individual letter tokens</li>
    <li style="margin-left:10px">Traverse from left to right of the corpus</li>
    <li style="margin-left:10px">At each iteration, analyze a <b><font color="green">pair</font> of single tokens <b>("h", "##u" from the word "hugging")</b> </li>
    <li style="margin-left:10px">Record the score of this pair in a table</li>
    <li style="margin-left:10px">Record all such score of sequential pairs in the traversed corpus</li>
    <li style="margin-left:10px">Select the highest scoring potential pair, suppose : <b>("l", "##e" from the words "learner" or "learn")</b></li>
    <li style="margin-left:10px">Join the pair <b>("l", "##e" -> "le")</b></li>
    <li style="margin-left:10px">Replace all cooccurrences of "l" and "##e" with "le" in the vocabulary</b></li>
    <li style="margin-left:10px">Repeat the steps of scoring and updating until reaching the vocabulary size</b></li>
<ul> 

<div>
    <img src="img/l2_wordpiece_2.jpg" width=1000px>   
</div>

<p style="background-color:#faecac; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">Let’s say we have the word "walker" and the subwords "wa", "lk", "er" and we are at a point where we need to decide what to merge.
Suppose ["wa"] appears frequently, but ["wa"] and ["lk"] together aren't as frequent.
On the other hand, ["lk"] and ["er"] occur together often, e.g., in words like "walker", "talker", "stalker".
Based on the scoring mechanism, the tokenizer will prioritize merging ["lk"] and ["er"] into ["lker"] (or eventually ["walk", "##er"]), because this pair maximizes the likelihood of the text, compared to merging ["wa"] and ["lk"].

<p style="background-color:orange; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> The real_wordpiece library creates conflict with the existing tokenizers version as it is an older implementation and is <b>almost obsolete</b></p>

In [47]:
from real_wordpiece.trainer import RealWordPieceTrainer
from tokenizers.models import WordPiece

real_wordpiece_tokenizer = Tokenizer(WordPiece())
real_wordpiece_tokenizer.pre_tokenizer = Whitespace()

# The training algorithm is different than the Huggingface's WordPiece
real_wordpiece_trainer = RealWordPieceTrainer(vocab_size=27)

In [48]:
real_wordpiece_trainer.train_tokenizer(training_data, real_wordpiece_tokenizer)
real_wordpiece_tokenizer.get_vocab()

{'##n': 10,
 'd': 15,
 'a': 5,
 '##ed': 23,
 '##a': 1,
 'o': 16,
 'l': 6,
 '##r': 7,
 '##k': 3,
 '##g': 11,
 'n': 17,
 '##d': 8,
 'wa': 24,
 'lo': 19,
 '##o': 9,
 'r': 14,
 'walk': 26,
 '##er': 22,
 '##lk': 25,
 '##ng': 20,
 'w': 0,
 '##e': 4,
 '##l': 2,
 'k': 12,
 'g': 18,
 'long': 21,
 'e': 13}

In [49]:
real_wordpiece_tokenizer.encode("walker walked a long walk").tokens

['walk', '##er', 'walk', '##ed', 'a', 'long', 'walk']

In [50]:
real_wordpiece_tokenizer.encode("wlk").tokens

['w', '##lk']

**Unknown Characters:**
The following line will produce an error because it contains unknown characters. Please uncomment the line and run it to see the error.

In [51]:
real_wordpiece_tokenizer.encode("she walked").tokens

Exception: WordPiece error: Missing [UNK] token from the vocabulary

<p style="background-color:#faecac; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">The prime difference between <b>RealWordPiece</b> and <b>Huggingface's WordPiece</b> is that the real one uses the <b>scoring function</b> while pairing to effectively detect prefix and suffixes. This enables the tokenization of new words into proper <b>base word and suffixes</b>. But the Huggingface one pairs the base tokens based on frequency of occurrences. <br>
"walker" -> Huggingface -> "walker" (Largest frequently occurring token dominates)<br>
"walker" -> RealWordPiece -> <b><font color="green">"walk"</font></b> "##er" (Base word and suffix dominate) <br>

## HuggingFace WordPiece and special tokens

We can include an **UNK** token to handle unknown tokens in our vocab. After training, whenever any given text has any new text in it which is not part of the **trained vocabulary**, it matches with the **UNK** in the vocab then and tokenizes accordingly  
In the real wordpiece

In [52]:
from tokenizers.trainers import WordPieceTrainer

unk_token = "[UNK]"

wordpiece_model = WordPiece(unk_token=unk_token)
wordpiece_tokenizer = Tokenizer(wordpiece_model)
wordpiece_tokenizer.pre_tokenizer = Whitespace()
wordpiece_trainer = WordPieceTrainer(vocab_size=28, special_tokens=[unk_token]) # 27 + 1 for the new UNK token 

In [53]:
wordpiece_tokenizer.train_from_iterator(
    training_data, 
    wordpiece_trainer
)
wordpiece_tokenizer.get_vocab()

{'##e': 17,
 'n': 7,
 '##g': 16,
 'walk': 22,
 'l': 6,
 '##a': 11,
 'd': 2,
 '##d': 19,
 'o': 8,
 '##l': 12,
 '##ng': 25,
 'g': 4,
 'r': 9,
 '##k': 13,
 '##n': 15,
 '##lk': 21,
 'w': 10,
 '##r': 18,
 'lo': 24,
 'walked': 27,
 'a': 1,
 'e': 3,
 'walke': 23,
 '##o': 14,
 'k': 5,
 'walker': 26,
 '[UNK]': 0,
 'wa': 20}

In [54]:
wordpiece_tokenizer.encode("walker walked a long walk").tokens

['walker', 'walked', 'a', 'lo', '##ng', 'walk']

In [55]:
wordpiece_tokenizer.encode("wlk").tokens

['w', '##lk']

In [56]:
wordpiece_tokenizer.encode("she walked").tokens

['[UNK]', 'walked']

## Unigram

<ul style="background-color:#edfbff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    <li style="margin-left:10px">Break down the corpus into all possible list of <b>distinct</b> tokens</li>
    <li style="margin-left:10px">Starts with a huge vocabulary (the entire list of possible tokens)</li>
    <li style="margin-left:10px">Each token has a likelihood, which is calculated using the <b>frequency</b> of occurrences of that token in the corpus</li>
    <li style="margin-left:10px">The corpus likelihood is the product of the probabilities of all the subword tokens in the corpus</li>
    <li style="margin-left:10px">This likelihood is maximized by keeping the subwords that contribute most to this probability while pruning low-probability subwords</li>
    <li style="margin-left:10px">At each iteration, the token with the <b><font color="green">least reduction in total likelihood of the corpus</font> is removed from the vocab</li>
    <li style="margin-left:10px">We keep on reducing and recalculate the corpus likelihood until we reach <b><= vocab size</b></li>
<ul> 

<table><tr>
<td> <img src="img/l2_unigram_1.jpg" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="img/l2_unigram_2.jpg" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>

In [57]:
from tokenizers.trainers import UnigramTrainer
from tokenizers.models import Unigram

unigram_tokenizer = Tokenizer(Unigram())
unigram_tokenizer.pre_tokenizer = Whitespace()
unigram_trainer = UnigramTrainer(
    vocab_size=14, 
    special_tokens=[unk_token],
    unk_token=unk_token,
)

unigram_tokenizer.train_from_iterator(training_data, unigram_trainer)
unigram_tokenizer.get_vocab()

{'o': 9,
 '[UNK]': 0,
 'r': 11,
 'd': 8,
 'w': 3,
 'l': 5,
 'walke': 1,
 'g': 10,
 'n': 12,
 'e': 7,
 'k': 2,
 'a': 6,
 'walk': 4}

In [58]:
unigram_tokenizer.encode("walker walked a long walk").tokens

['walke', 'r', 'walke', 'd', 'a', 'l', 'o', 'n', 'g', 'walk']

In [59]:
unigram_tokenizer.encode("wlk").tokens

['w', 'l', 'k']

In [60]:
unigram_tokenizer.encode("she walked").tokens

['sh', 'e', 'walke', 'd']

In [61]:
unigram_tokenizer.encode("she walked").ids

[0, 7, 1, 8]

In [63]:
unigram_tokenizer.encode("they").ids

[0, 7, 0]

In [65]:
unigram_tokenizer.encode("they").tokens

['th', 'e', 'y']

We can see in the above example that the **"they"** has only **"e"** as common subword token from the existing vocab, and the parts **"th"** and **"y"** falls into **[UNK]** token. So the ids of tokens **"th" & "y"** show **0**, referring to **[UNK]**

In [64]:
unigram_tokenizer.get_vocab()

{'o': 9,
 '[UNK]': 0,
 'r': 11,
 'd': 8,
 'w': 3,
 'l': 5,
 'walke': 1,
 'g': 10,
 'n': 12,
 'e': 7,
 'k': 2,
 'a': 6,
 'walk': 4}

<p style="background-color:#faecac; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">The key feature of unigram language modelling tokenizer is that it does not populate any unnecessary <b>glitch tokens</b> like "##alk" or "##al" or "##lk" which do not make any sense for our training corpus

## SentencePiece

<p style="background-color:#faecac; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> This is the same as other algorithms but also consider whitespaces while building the vocabulary. Sometimes we need to consider words like <b>"Real Madrid"</b> or <b>San Fransisco</b> as a single token. Sentencepiece can deal with these cases

<div>
    <img src="img/l2_sentencepiece_1.png" width=400px>    
</div>